In [1]:
# Install required Python packages
!pip install -q sentence-transformers chromadb pandas pyyaml openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.6 MB/s eta 

In [2]:
# Download and install .NET SDK
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --version 8.0.100

--2025-08-18 19:04:24--  https://dot.net/v1/dotnet-install.sh
Resolving dot.net (dot.net)... 20.76.201.171, 20.112.250.133, 20.236.44.162, ...
Connecting to dot.net (dot.net)|20.76.201.171|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://builds.dotnet.microsoft.com/dotnet/scripts/v1/dotnet-install.sh [following]
--2025-08-18 19:04:25--  https://builds.dotnet.microsoft.com/dotnet/scripts/v1/dotnet-install.sh
Resolving builds.dotnet.microsoft.com (builds.dotnet.microsoft.com)... 23.219.78.69, 23.219.78.79, 2600:1409:9800:e3::172e:e40a, ...
Connecting to builds.dotnet.microsoft.com (builds.dotnet.microsoft.com)|23.219.78.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘dotnet-install.sh’

dotnet-install.sh       [ <=>                ]  62.15K  --.-KB/s    in 0.01s   

2025-08-18 19:04:25 (4.81 MB/s) - ‘dotnet-install.sh’ saved [63644]

dotnet-install: Attempting

In [3]:
# Install Roslynator CLI
!~/.dotnet/dotnet tool install -g Roslynator.DotNet.Cli

=========
Welcome to .NET 8.0!
---------------------
SDK Version: 8.0.100

Telemetry
---------
The .NET tools collect usage data in order to help us improve your experience. It is collected by Microsoft and shared with the community. You can opt-out of telemetry by setting the DOTNET_CLI_TELEMETRY_OPTOUT environment variable to '1' or 'true' using your favorite shell.

Read more about .NET CLI Tools telemetry: https://aka.ms/dotnet-cli-telemetry

----------------
Installed an ASP.NET Core HTTPS development certificate.
To trust the certificate, view the instructions: https://aka.ms/dotnet-https-linux

----------------
Write your first app: https://aka.ms/dotnet-hello-world
Find out what's new: https://aka.ms/dotnet-whats-new
Explore documentation: https://aka.ms/dotnet-docs
Report issues and find source on GitHub: https://github.com/dotnet/core
Use 'dotnet --help' to see available commands or visit: https://aka.ms/dotnet-cli
----------------------------------------------------

In [4]:
# Update PATH using os.environ (so Python and subprocesses can use it)
import os
os.environ["DOTNET_ROOT"] = "/root/.dotnet"
os.environ["PATH"] += os.pathsep + "/root/.dotnet"
os.environ["PATH"] += os.pathsep + "/root/.dotnet/tools"

In [5]:
import shutil
print("dotnet path:", shutil.which("dotnet"))
print("roslynator path:", shutil.which("roslynator"))

dotnet path: /root/.dotnet/dotnet
roslynator path: /root/.dotnet/tools/roslynator


In [16]:
import glob
import subprocess
import re
from IPython.display import clear_output
import time
from sentence_transformers import SentenceTransformer
import chromadb
import uuid
import os
import openai
from chromadb import Client
from chromadb.config import Settings
from openai import OpenAI
import json

ModuleNotFoundError: No module named 'chromadb'

In [7]:
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [8]:
import json

secret_file_path = "/content/drive/My Drive/Secret_Keys/OpenAI_Secret_Key.json"

with open(secret_file_path, "r") as f:
  data=json.load(f)

secret_Key = data["open_ai_key"]


In [9]:
github_pat_token_file_path =  "/content/drive/My Drive/Secret_Keys/github_personal_access_token.json"

with open(github_pat_token_file_path, "r") as f:
  data=json.load(f)

GITHUB_PAT = "gittoken" #data["github_pat_token"]

In [1]:
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.dotnet/tools")
GITHUB_REPO_URL = "https://github.com/TheBrickLens/TheBrickLens.UserManagement.git"
COLAB_REPO_PATH = "/content/TheBrickLens.UserManagement"

NameError: name 'os' is not defined

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_chroma_client():
  if "CHROMA_CLIENT" not in globals():
    globals()["CHROMA_CLIENT"] = Client(Settings(
        persist_directory = "/content/chroma_cs_issues",
        anonymized_telemetry = False
    ))
  return globals()["CHROMA_CLIENT"]

chroma_client = get_chroma_client()

colleciton = chroma_client.get_or_create_collection(name="roslynator_issues")

In [ ]:
os.environ["OPENAI_API_KEY"] = secret_Key
client= OpenAI()

In [ ]:
def clone_private_repo():
  repo_name = GITHUB_REPO_URL.rstrip("/")[-1].replace(".git", "")
  repo_path = os.path.join("/content", repo_name)

  if os.path.exists(repo_path):
    print(f"Repo already exists at: {repo_path}")
    return repo_path

  safe_url = GITHUB_REPO_URL.replace("https://",f"https://{GITHUB_PAT}@")

  env = os.environ.copy()
  env["GIT_ASKPASS"] = "echo"
  env["GIT_TERMINAL_PROMPT"] = "0"

  try:
    subprocess.run(
        ["git", "clone", safe_url, repo_path],
        check = True,
        capture_output = True,
        text= True,
        env = env
    )
    print(f"Successfully clone to: {repo_path}")
  except subprocess.CalledProcessError as e:
    print("Git clone failed")
    print("STDOUT:\n", e.stdout)
    print("STDERR:\n", e.stderr)
    raise

  return repo_path

In [ ]:
def extract_cs_files(repo_path):
  cs_files = {}

  for root, _, files in os.walk(repo_path):
    for file in files:
      if file.endswith(".cs"):
        full_path = os.path.join(root, file)
        try:
          with open(full_path,"r",encoding = "utf-8") as f:
            cs_files[full_path] = f.read()
        except Exception as e:
          print(f"Failed to read {full_path}: {e}")

  return cs_files

In [ ]:
def run_roslynator_on_file(file_path):
  result = subprocess.run(
        ["roslynator", "analyze", file_path],
        stdout= subprocess.PIPE,
        stderr= subprocess.PIPE,
        text= True
    )

  if result.stderr.strip():
    print(f"Roslynator stderr for {file_path}:\n{result.stderr}")

  return result.stdout

In [ ]:
def parse_roslynator_output(output,file_path):
  issues = []
  pattern = re.compile(r"^(.*\.cs)\((\d+),\d+\):\s+(RCS\d+):\s+(.*)$")

  for line in output.strip().splitlines():
    match = pattern.match(line.strip())
    if match:
      issues.append({
          "file": match.group(1),
          "line": int(match.group(2)),
          "id": match.group(3),
          "message": match.group(4)
      })

  return issues

In [ ]:
def analyze_all_cs_files(cs_files):
  all_issues = []

  for file in cs_files:
    output = run_roslynator_on_file(file)
    file_issues = parse_roslynator_output(output, file)
    all_issues.extend(file_issues)

  print(f"Detected {len(all_issues)} total issues")
  return all_issues

In [ ]:
def store_issues_in_chromadb(issues,embeddings,chroma_client):
  if not issues or not embeddings.any():
    raise ValueError("No issues or embeddings to store")

  collection = chroma_client.get_or_create_collection(name="roslynator_issues")

  for i, issue in enumerate(issues):
    metadata = {
          "file": issue["file"],
          "line": issue["line"],
          "id": issue["id"],
          "message": issue["message"],
          "severity": issue["severity"]
    }

    document_text = (
        f"This is code analysis issue with ID {issue['id']}. "
        f"It occures in the file {issue['file']} on line {issue['line']}"
        f"The sevirity of the issue is {issue['severity']}"
        f"The message describing the issue is {issue['message']}."
        f"This might relate to C# compiler warnings or best practices violation. "
    ).lower()

    collection.add(
        documents = [document_text],
        metadatas= [metadata],
        ids = [str(uuid.uuid4())],
        embeddings = embeddings[i]
    )

In [ ]:
def show_analysis_report(issues, limit =50):
  print(f"Total issues: {len(issues)}\n")

  for i, issue in enumerate(issues):
    print(f"{i+1: >3 }.[{issue['id']}] Line{issue['line']} in {os.path.basename(issue['file'])}")

    print (f"    | issue: {issue['message']}")

  if len(issues) > limit:
    print(f"... and {len (issues) - limit } more issues.")

In [ ]:
def query_code_issues(query, collection, embed_model, top_k=5):
  query = query.strip().lower()
  query_embedding = embed_model.encode([query]).tolist()

  results = colleciton.query(
      query_embeddings = query_embedding,
      n_result = top_k,
  )

  matches= []

  for i in range(len(results["documents"][0])):
    doc= results["documents"][0][i]
    metadata = results["metadatas"][0][i]
    matches.append((doc, metadata))

  return results

In [ ]:
def show_query_results(matches):
  if not matches:
    print("No matching issues found")
    return

  print(f"\n Top {len(matches)} matching issues: \n")
  for i, (doc, meta) in enumerate(matches):
    print(f"{i+1: >3 }.[{meta['id']}] Line{meta['line']} in {os.path.basename(meta['file'])}")

In [ ]:
def get_code_context(file_path, line_number, context_lines = 5):
  with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
  start = max(0, line_number - context_lines - 1)
  end = min(len(lines), line_number + context_lines)
  return "".join(lines[start:end]), lines

In [ ]:
def suggest_fix_with_gpt(issue, code_snippet):
  prompt = f"""You are an expert C# code refactoring assistant.
  You are fixing this issue flagged by Roslynator:
  Issue: {issue['message']}
  File: {os.path.basename(issue['file'])}
  Line: {issue['line']}
  Code: {code_snippet}
  Suggest the complete **replacement code** (only that block), keeping it valid C#. Do not add explanations."""
  response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
  )
  return response.choices[0].message.content.strip()

In [ ]:
def auto_refactor_code(issues):
  fixed_files = {}
  for issue in issues:
    file_path = issue["file"]
    line_number = issue["line"]
    print("=" * 60)
    print(f"[{issue['id']}] in {file_path} at line {line_number}")
    print(f"Issue: {issue['message']}")
    code_context, lines = get_code_context(file_path, line_number)
    print("\nCode context:\n", code_context)
    suggestion = suggest_fix_with_gpt(issue, code_context)
    print("\nSuggested fix:\n", suggestion)
    choice = input("Apply fix? (y/n): ").strip().lower()
    if choice != "y":
      continue
    # Determine lines to replace (safely)
    start = max(0, line_number - 6)
    end = min(len(lines), line_number + 5)
    suggested_lines = [line + "\n" for line in suggestion.strip().splitlines()]
    lines[start:end] = suggested_lines

    # Save the updated file back to local repo
    with open(file_path, "w") as f:
      f.writelines(lines)
    fixed_files[file_path] = True
    print("Fix applied and file saved.\n")

  return fixed_files

In [ ]:
def parse_roslynator_report(report_text, repo_path):
  issues = []
  pattern = re.compile(r"^(.*\.cs)\((\d+),(\d+)\):\s+(\w+)\s+(.*)$")
  for line in report_text.strip().splitlines():
    match = pattern.match(line.strip())
    if match:
      file_path, line_num, col_num, rule_id, message = match.groups()
      file_path = os.path.normpath(os.path.join(repo_path, file_path.strip()))
      issues.append({
        "file": file_path,
        "line": int(line_num),
        "col": int(col_num),
        "id": rule_id.strip(),
        "message": message.strip(),
        "severity": "info"
      })
  return issues

In [ ]:
def query_issues_from_chromadb(query_text, colleciton_name= "roslynator_issue", top_k=5):
  client = CHROMA_CLIENT
  try:
    collection = client.get_or_create_collection(name= colleciton_name)
  except Exception as e:
    print (f"Failed to access Chromadb collection:{e}")
    return

  model = SentenceTransformer ("all-MiniLM-L6-v2")

  query_embedding = model.encode ([query_text]).tolist()

  results = colleciton.query(
      query_embeddings = query_embedding,
      n_result = top_k,
  )

  if not results or not results["documents"]:
    print(f"No matching issues found")
    return

  print(f"\nTop {top_k} matching issues for: '{query_text}'")
  for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print("-" * 40)
    print(f"File       : {meta['file']}")
    print(f"Line       : {meta['line']}")
    print(f"Issue ID   : {meta['id']}")
    print(f"Message    : {doc}")


In [ ]:
def get_files_with_issues(issues, cs_files):
  affected_file_names = set(os.path.basename(issues["file"]) for issues in issues)

  print(f"Affected filename from issues:", affected_file_names)

  all_cs_filenames = set(os.path.basename(p) for p in cs_files.keys())
  print("Available C# files in repo: ", all_cs_filenames)

  matching_files = {
      path:
      content for path, content in cs_files.items() if os.path.basename(path) in affected_file_names
  }

  print("Matched filenames: ", list(matching_files.keys()))
  return matching_files

In [ ]:
def process_and_fix_issues(issues, files_to_fix, model, fixed_files, state):
    for issue in issues:
        reported_path = issue["file"]
        filename_only = os.path.basename(reported_path)
        line = issue["line"]
        message = issue["message"]
        # Try suffix match
        match_path = next(
            (path for path in files_to_fix if path.endswith(reported_path)),
            None
        )
        # Fallback: match by filename
        if not match_path:
            candidates = [path for path in files_to_fix if os.path.basename(path) == filename_only]
            if len(candidates) == 1:
                match_path = candidates[0]
            elif len(candidates) > 1:
                print(f"Multiple matches for {filename_only}. Skipping.")
                continue
        if not match_path:
            print(f"Skipping: no match for {reported_path}")
            continue
        code = files_to_fix[match_path]

        # 1. Header: Project + File
        project_name = match_path.split("/")[1] if "/" in match_path else "UnknownProject"
        print(f"\nProject: {project_name} | File: {match_path}")

        # 2. Show the problem
        print(f"Issue at line {line}: {message}")

        # 3. Separator
        print("=" * 80)

        # 4. Prompt for fix
        prompt = f"""The following C# file has an issue at line {line}:
        Filename: {match_path}
        Issue: {message}
        Here is the full content of the file:
        {code}
        Provide the complete corrected version of this file with only the required fix applied. Do not make unrelated changes."""
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
            )
            fixed_code = response.choices[0].message.content.strip()
        except Exception as e:
            print("OpenAI error:", e)
            continue

        # 5. Show GPT fix
        print("Suggested Fix:")
        print(fixed_code[:2000])
        # 6. Final separator
        print("=" * 80)
        # 7. Prompt
        choice = input("Apply this fix? (yes/no): ").strip().lower()
        if choice == "yes":
            fixed_files[match_path] = fixed_code
            print(f"Fix applied to: {match_path}")
        else:
            print("Skipped.")
        input("Press Enter to continue...")
        clear_output()

    print(f"Total issues attempted: {len(issues)}")
    print(f"Total fixes recorded: {len(fixed_files)}")

    if fixed_files:
        maybe_commit_changes(state["repo_path"], fixed_files)

In [ ]:
def write_fixed_files_to_disk(fixed_files, base_path):
  for filename, updated_code in fixed_files.items():
    full_path = os.path.join(base_path, filename)
    try:
      with open(full_path, "w", encoding="utf-8") as f:
        f.write(updated_code)
      print(f"Updated: {filename}")
    except Exception as e:
      print(f"Failed to write: {filename} — {e}")



In [ ]:
def maybe_commit_changes(repo_path, fixed_files):
  if not fixed_files:
    print("No files were modified. Skipping commit.")
    return
  commit = input("\nDo you want to commit the changes to the local repo? (yes/no): ").strip().lower()
  if commit != "yes":
    print("Skipped committing.")
    return
  commit_msg = input("Enter commit message: ").strip()
  try:
    subprocess.run(["git", "add", "."], cwd=repo_path, check=True)
    subprocess.run(["git", "commit", "-m", commit_msg], cwd=repo_path, check=True)
    print(f"Successfully committed with message: {commit_msg}")
  except subprocess.CalledProcessError as e:
    print("Commit failed.")
    print("STDOUT:\n", e.stdout)
    print("STDERR:\n", e.stderr)

In [2]:
def embed_issues(issues, model_name = "all-MiniLM-L6-v2"):
  if not issues:
    raise ValueError("No issues to embed")

  model = SentenceTransformer(model_name)
  texts = [f"{i['file']} - {i['id']} - {i['message']}" for i in issues]
  return model.encode(texts, convert_to_numpy = True)

In [3]:
def run_roslynator_analysis(csproj_path):
  # Use the directory containing the dotnet executable
  msbuild_path = os.path.expanduser("~/.dotnet")
  if not os.path.exists(msbuild_path):
    raise FileNotFoundError(f"MSBuild directory {msbuild_path} does not exist. Ensure .NET SDK is installed.")
  command = [
    "roslynator", "analyze", csproj_path,
    "--msbuild-path", msbuild_path,
    "--verbosity", "d"
  ]
  print("Running Roslynator CLI...")
  result = subprocess.run(command, capture_output=True, text=True)
  if result.returncode not in (0, 1):  # 1 = warnings (Roslynator sometimes exits with 1 for diagnostics)
    print("Roslynator analysis failed.")
    print("STDERR:\n", result.stderr)
    print("STDOUT:\n", result.stdout)
    raise RuntimeError("Roslynator analysis failed with exit code:", result.returncode)
  print("Roslynator analysis completed.")
  return result.stdout

In [4]:
def find_csproj_files(repo_path):
  csproj_files = []

  for root, dirs, files in os.walk(repo_path):
    for file in files:
      if file.endswith(".csproj"):
        csproj_files.append(os.path.join(root, file))

  return csproj_files

In [5]:
def restore_packages(csproj_path):
  result = subprocess.run(
      ["dotnet", "restore", csproj_path],
      capture_output =True,
      text = True
  )

  if result.returncode != 0:
    print("Package restore failed")
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    raise RuntimeError("Package restore failed")
  else:
    print("Package restore completed.")

In [6]:
def print_menu():
  print("=" * 50)
  print("C# AUTO-REFACTOR AGENT")
  print("=" * 50)
  print("1. Clone GitHub Repo")
  print("2. Run Roslynator Analysis + Store")
  print("3. Show Analysis Report")
  print("4. Query Code Issues")
  print("5. Auto-Refactor Code")
  print("6. Exit")
  print("=" * 50)

In [7]:
def handle_show_report(state):
  if not state["all_issues"] and is_analysis_available_in_chromadb():
    try:
      collection = chroma_client.get_collection("roslynator_issues")
      results = collection.get(include=["metadatas"])
      state["all_issues"] = [
        {
          "file": meta["file"],
          "line": int(meta["line"]),
          "id": meta["id"],
          "message": meta["message"],
          "severity": meta.get("severity", "info")
        } for meta in results["metadatas"]
      ] if results["metadatas"] else []
    except Exception as e:
      print(f"Error retrieving issues from ChromaDB: {e}")
      state["all_issues"] = []
  if not state["all_issues"]:
    print("Please run the analysis first.")
    time.sleep(2)
    return
  show_analysis_report(state["all_issues"])

In [8]:
def handle_query_issues():
  if not is_analysis_available_in_chromadb():
    print("Please run the analysis first.")
    time.sleep(2)
    return
  query = input("Enter query about code issues: ").strip()
  matches = query_issues_from_chromadb(query)
  show_query_results(matches)
  input("Press Enter to continue...")

In [9]:
def handle_show_analysis():
  if not is_analysis_available_in_chromadb():
    print("Please run the analysis first.")
    time.sleep(2)
    return
  show_analysis_report()
  input("Press Enter to continue...")

In [10]:
def handle_auto_refactor(state):
  repo_path = state.get("repo_path")
  if not repo_path and "COLAB_REPO_PATH" in globals():
    repo_path = globals().get("COLAB_REPO_PATH")
  if not repo_path or not os.path.exists(os.path.join(repo_path, ".git")):
    print("Repository path not set or invalid. Please clone the repository first (option 1).")
    time.sleep(2)
    return
  state["repo_path"] = repo_path
  state["repo_cloned"] = True
  if not state["all_issues"]:
    try:
      collection = chroma_client.get_collection("roslynator_issues")
      results = collection.get(include=["metadatas"])
      state["all_issues"] = [
        {
          "file": meta["file"],
          "line": int(meta["line"]),
          "id": meta["id"],
          "message": meta["message"],
          "severity": meta.get("severity", "info")
        } for meta in results["metadatas"]
      ] if results["metadatas"] else []
    except Exception as e:
      print(f"Error retrieving issues from ChromaDB: {e}")
      state["all_issues"] = []
  if not state["all_issues"]:
    print("Please run the analysis first.")
    time.sleep(2)
    return
  cs_files = extract_cs_files(state["repo_path"])
  files_to_fix = get_files_with_issues(state["all_issues"], cs_files)
  process_and_fix_issues(
    state["all_issues"],
    files_to_fix,
    model="gpt-4o-mini",
    fixed_files=state["fixed_files"],
    state=state
  )
  if state["fixed_files"]:
    write_fixed_files_to_disk(state["fixed_files"], state["repo_path"])
    maybe_commit_changes(state["repo_path"], state["fixed_files"])
    state["issues_fixed"] = True
  else:
    print("No fixes were applied.")
    time.sleep(2)

In [11]:
def is_analysis_available_in_chromadb():
  try:
    collection = chroma_client.get_collection("roslynator_issues")
    return collection.count() > 0
  except Exception as e:
    print(f"Error accessing ChromaDB: {e}")
    return False

In [12]:
def handle_clone_repo(state):
    repo_path = clone_private_repo()
    if os.path.exists(repo_path):
        state["repo_path"] = repo_path
        state["repo_cloned"] = True
        print(f"Repository cloned to: {repo_path}")
    else:
        print(f"Failed to clone repository to: {repo_path}")
        state["repo_path"] = None
        state["repo_cloned"] = False
    time.sleep(2)

In [13]:
def handle_run_analysis(state):
    # Use defined COLAB_REPO_PATH or fallback to repo path
    repo_path = globals().get("COLAB_REPO_PATH", state.get("repo_path"))
    if not repo_path or not os.path.exists(os.path.join(repo_path, ".git")):
        print(f"Valid repository not found at {repo_path}. Please clone the repository first (option 1).")
        time.sleep(2)
        return
    state["repo_path"] = repo_path
    state["repo_cloned"] = True
    csproj_files = find_csproj_files(repo_path)
    if not csproj_files:
        print(f"No .csproj files found in {repo_path}.")
        time.sleep(2)
        return
    state["all_issues"] = []
    for csproj_path in csproj_files:
        print(f"Running Roslynator on: {csproj_path}")
        try:
            restore_packages(csproj_path)
            report_text = run_roslynator_analysis(csproj_path)
            issues = parse_roslynator_report(report_text, repo_path)
            project_dir = os.path.dirname(csproj_path)
            for issue in issues:
              issue["file"] = os.path.normpath(os.path.join(project_dir, issue["file"]))
            state["all_issues"].extend(issues)
        except Exception as e:
            print(f"Error analyzing {csproj_path}: {e}")
    if not state["all_issues"]:
        print("No issues detected by Roslynator.")
        time.sleep(2)
        return
    embeddings = embed_issues(state["all_issues"])
    store_issues_in_chromadb(state["all_issues"], embeddings, chroma_client)
    state["analysis_done"] = True
    print(f"Stored {len(state['all_issues'])} issues in ChromaDB.")
    time.sleep(2)

In [14]:
def interactive_code_analysis():
  state = {
    "repo_path": None,
    "all_issues": [],
    "fixed_files": {},
    "issues_fixed": False,
  }
  while True:
    print_menu()
    choice = input("Select an option: ").strip()
    if choice == "1":
      handle_clone_repo(state)
    elif choice == "2":
      handle_run_analysis(state)
    elif choice == "3":
      handle_show_report(state)
    elif choice == "4":
      handle_query_issues()
    elif choice == "5":
      handle_auto_refactor(state)
    elif choice == "6":
      break
    else:
      print("Invalid choice. Try again.")
      time.sleep(1)

In [15]:
interactive_code_analysis()

C# AUTO-REFACTOR AGENT
1. Clone GitHub Repo
2. Run Roslynator Analysis + Store
3. Show Analysis Report
4. Query Code Issues
5. Auto-Refactor Code
6. Exit
Select an option: 1


NameError: name 'clone_private_repo' is not defined